## Get all csv headers

In [11]:
from numpy import genfromtxt
import pandas as pd
from os import listdir
from os.path import isfile, join

mypath = "../data/csvs_per_year/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
all_columns = []

for filename in onlyfiles[1:]:
    with open(join(mypath,filename)) as f:
        columns = f.readline().split(',')
        all_columns += columns

In [6]:
sorted(set(all_columns))

['BEN',
 'CO',
 'EBE',
 'MXY',
 'NMHC',
 'NO',
 'NO_2',
 'NOx',
 'OXY',
 'O_3',
 'PM10',
 'PM25',
 'PXY',
 'SO_2',
 'TCH',
 'TOL',
 'date',
 'station\n']

## Import

In [8]:
import sqlite3
conn = sqlite3.connect('../data/air_pollution-new.db')
c = conn.cursor()

### Import stations

In [46]:
df=pd.read_csv('../data/stations.csv', sep=',')
for index, row in df.iterrows():
    # Insert a row of data
    c.execute("INSERT INTO stations (id, name, address, lat, lon, elevation) VALUES (?,?,?,?,?,?)", (row['id'], row['name'], row['address'], row['lat'], row['lon'], row['elevation']))

# Save (commit) the changes
conn.commit()

# c.execute('SELECT * FROM stations')
# print(c.fetchall())

# # We can also close the connection if we are done with it.
# # Just be sure any changes have been committed or they will be lost.
# # conn.close()

## Import measurements

In [9]:
def add_all_columns(df):
    for column in all_columns:
        if column not in df:
            df[column] = None

In [15]:
for filename in onlyfiles:
    print("Reading " + filename)
    df = pd.read_csv(join(mypath,filename), sep=',')
    add_all_columns(df)
    print("Inserting " + filename)
    for index, row in df.iterrows():
        c.execute('''
        INSERT INTO
            measurements
            (station_id, date, BEN, CO, EBE, MXY, NMHC, NO, NO_2, NOx, OXY, O_3, PM10, PM25, PXY, SO_2, TCH, TOL)
        VALUES
            (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''',
            (
                row['station'],
                row['date'],
                row['BEN'],
                row['CO'],
                row['EBE'],
                row['MXY'],
                row['NMHC'],
                row['NO'],
                row['NO_2'],
                row['NOx'],
                row['OXY'],
                row['O_3'],
                row['PM10'],
                row['PM25'],
                row['PXY'],
                row['SO_2'],
                row['TCH'],
                row['TOL']
            ))
    conn.commit()

Reading madrid_2018.csv
Inserting madrid_2018.csv
